In [ ]:
## Bevor wir anfangen laden wir ein paar Bibliotheken:

In [ ]:
#module includes
from ipywidgets import interact, interactive_output, interact_manual, interactive, FloatSlider, IntSlider, Dropdown
import matplotlib
from IPython.display import display
from math import sin, cos, exp, log, sqrt
from ngsolve import *
from mesh1d import *
from draw1d import * 
from specials import *

# Einführung in die Finite-Elemente-Methode für das 1D Wärmeleitungsproblem

- Die Differenzialgleichung:
$$
  - k \frac{d^2 u}{d x^2}(x) = q_Q(x) \quad \text{für alle} \quad x \quad \text{ im Intervall } (a,b).
$$

- Die Randbedingungen:
  - Vorgegebene Temperatur am Rand, z.B: 
$$
  u(a) = T_a \in \mathbb{R}, \qquad u(b) = T_b \in \mathbb{R}. 
$$
  - Vorgegebener Wärmefluss am Rand, z.B: 
$$
  k \frac{ d u}{d x}(a) = q_S^a \in \mathbb{R}, \qquad - k \frac{ d u}{d x}(b) = q_S^b \in \mathbb{R}.
$$

## Unbekannte Größe ist eine Funktion

- Die Lösung der Differenzialgleichung ist eine Funktion 
- Eine Funktion hat "unendlich viele Freiheitsgrade", Beispiel:
    - Sei f(x) eine Funktion im Intervall [0,1]
    - Wir fixieren N+1 Freiheitsgrade an N+1 Stellen: f(x) = 0 für x = $\frac{0}{N}, \frac{1}{N}, \frac{2}{N}, .. , \frac{N-1}{N}, \frac{N}{N}$
    - Dennoch existieren Funktionen f(x) die nicht überall 0 sind:

In [ ]:
interact(DrawSpecialFunction,N=(1,193,12));

- Die betrachteten Funktionen sind für $N \to \infty$ nicht mehr "glatt"
- Wenn die gesuchte Funktion "glatt" ist, kann die Funktion mit endlich vielen Freiheitsgraden gut approximiert werden:

In [ ]:
align(interactive(DrawErrorToSmoothFunction,N=(10,100,10),smooth_function=["sin(3*pi*x)*cos(5*pi*x)",
                     "x**8+6*x**4","sin(8*pi*x)"]))

## Grundidee Diskretisierung partieller Differenzialgleichungen

 1. Wähle Darstellung der Funktion so, sodass die Funktion noch gut angenährt wird 
   
   (großes $N$ ist besser (genauer))
   
 2. Bestimme Gleichungen für die $N$ Unbekannten
 
 3. Löse Gleichungen 
   
   (kleines $N$ ist besser (günstiger))


## 1. "Approximation" einer "glatten" Funktion

### Freiheitsgrade zur Approximation einer Funktion:
   $
       u_h(x) = c_0 \varphi_0(x) + c_1 \varphi_1(x) + ... + c_{n-1} \varphi_{n-1}(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)
   $
   - Die Koeffizienten $c_i$ sind die Unbekannten ("Freiheitsgrade"), $i=0,..,n-1$.
   - $\varphi_i(x)$ sind die "Basisfunktionen".

In [ ]:
align(interactive(DrawBasisFunction, N=[4,6,8], i=IntSlider(min=0, max=41, step=1, continuous_update=False),
                  order=[1,2,3]))

Die Basisfunktionen erster Ordnung sind
$$
  \varphi_i(x) = \left\{ \begin{array}{cl} 1-N\left|x-\frac{i}{N}\right| & \text{ für } x \in \left[\frac{i-1}{N}, \frac{i+1}{N}\right], \\  & \text{ sonst. }  \end{array} \right.
$$
Die Basisfunktionen höherer Ordnung sind Polynome, die nur in einem Intervall von Null verschieden sind und auf den Rändern des Intervalls Null sind.

### Der Raum aller Funktionen der Form $u_h(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)$
 - Alle Funktionen, die als Linearkombination der Basisfunktionen dargestellt werden können, werden im 
 **"Finite-Elemente-Raum"** gesammelt:
 $$
   V_h \text{ ist die Menge aller Funktionen }u_h \text{ mit } u_h(x) = \sum_{i=0}^{n-1} c_i \varphi_i(x)
 $$

In [ ]:
align(interactive(ApproximateWithFESpace, 
                  N=[4,8,16,32,64], 
                  order=[1,2,3], 
                  f=["sin(3·pi·x)",
                     "sin(3·pi·x)·cos(5·pi·x)"]))

### Visualisierung der "partiellen Summe" $u_h^j = \sum_{i=0}^{\color{red}j} c_i \varphi_i(x)$:

In [ ]:
align(interactive(PartialApproximateWithFESpace, N=[4,8,16,32,64], 
                  order=[1,2,3], f=["sin(3·pi·x)","sin(3·pi·x)·cos(5·pi·x)"], j=IntSlider(min=-1, max=200, step=1, continuous_update=False)))

## 2. Bestimmung von Gleichungen für die Unbekannten

#### Ziel:

* Finde $n$ Gleichungen für die $n$ Unbekannten: 
* Da die Differenzialgleichung linear ist, erwarten wir, dass wir $n$ lineare Gleichungen finden können. 
* Wir suchen also zunächst ein lineares Gleichungssystem der Form 
 $$
 \left( \begin{array}{cccc} 
 A_{0,0} & A_{0,1} & \dots & A_{0,n-1} \\ 
 A_{1,0} & A_{1,1} & \dots & A_{1,n-1} \\ 
 \vdots  & \vdots & \ddots & \vdots \\ 
 A_{n-1,0} & A_{n-1,1} & \dots & A_{n-1,n-1} \\ 
 \end{array}
 \right) \cdot 
  \left( \begin{array}{c} 
 c_{0} \\ c_1 \\ \vdots \\ c_{n-1}
 \end{array}
 \right)
 =
  \left( \begin{array}{c} 
 f_{0} \\ f_1 \\ \vdots \\ f_{n-1}
 \end{array}
 \right)
  $$
  zur Bestimmung der Koeffizienten.
  
* Achtung: Nummerierung von $0$ bis $n-1$ (alternativ auch von $1$ bis $n$ möglich).

- Benötigen $n$ Gleichungen für $n$ Unbekannte
- Bei der Finite-Elemente-Methode fordern wir die PDGl. **nicht an allen Punkten in $(a,b)$**
- Stattdessen formulieren wir zunächst die PDGl. um:


\begin{align*}
    - k \frac{d^2 u}{d x^2}(x) &= q_Q(x) \text{ in } (a,b), \\
    \text{multipliziere mit (fast) beliebiger} & \text{ Funktion } v(x) \text{ und integriere über das Gebiet: } \\
    \Longrightarrow  \int_a^b - k \frac{d^2 u}{d x^2}(x) v(x) \ dx &= \int_a^b q_Q(x) v(x) \ dx
\end{align*}


Durch die bestimmte Integration fällt $x$ aus der Gleichung raus. Das ist so gewollt, da wir gerade nicht die unbekannte Funktion $u(x)$ mit unendlich vielen Freiheitsgraden bestimmen wollen (und können). Stattdessen wollen wir ein Gleichungssystem mit **endlich** vielen Unbekannten lösen.
Mit der (fast) beliebigen Wahl von $v(x)$ ist das noch nicht erreicht. Wir müssen später noch engere Bedingungen an die möglichen $v(x)$ stellen, um nur endlich viele Freiheitsgrade zu erhalten.

Wenn wir $u(x)$ als Summe von Basisfunktionen ausdrücken, ist für Basisfunktionen 1. Ordnung (die ja stückweise linear sind) die zweite Ableitung überall Null. Die linke Seite der Gleichung oben wäre also immer Null. Um das zu verhindern, wandeln wir die zweite Ableitung durch partielle Integration in eine erste Ableitung um:
* 
$\leadsto$ [Übung zum partiellen Integrieren und Randbedingungen](../../../Uebungen/blatt_zu_1DFEM.pdf)


$$
    \Longrightarrow  \int_a^b k \frac{d u}{d x}(x) \frac{d v}{d x}(x) \ dx - \Big[k \frac{d u}{dx} v(x) \Big]_a^b = \int_a^b q_Q(x) v(x) \ dx
    $$
   

* Der Einfachheit halber nehmen wir an das $u(a)$ und $u(b)$ gegeben sind (Dirichlet-Randbedingungen), d.h. 
$u(a)=T^a$, $u(b)=T^b$
* Die "Freiheit" die wir der unbekannten Funktion $u$ nehmen, nehmen wir auch der "Testfunktion" $v$ und setzen $v(a) = v(b) = 0$
* Damit vereinfacht sich das Problem zur (schwachen) Formulierung (FEM-Formulierung):
Finde die Funktion $u(x)$ mit $u(a)=T^a$, $u(b)=T^b$, sodass für (fast) jede Funktion $v(x)$ mit $v(a)=v(b)=0$ gilt
$$ \int_a^b k \frac{d u}{d x}(x) \frac{d v}{d x}(x) \ dx = \int_a^b q_Q(x) v(x) \ dx. $$
* Entsprechende modifizierte linke und rechte Seite bei anderen Randbedingungen (siehe Übung)

- Diskretisierung durch Forderung:

  Finde eine Funktion $u_h(x)$ mit $u_h = T^a \cdot \varphi_0 (x) + \sum_{\color{red}{j=1}}^{\color{red}{n-2}} c_j \varphi_j(x) + T^b \cdot \varphi_{n-1} (x)$, sodass 
  $$ \int_a^b k \frac{d u_h}{d x}(x) \frac{d v_h}{d x}(x) \ dx = \int_a^b q_Q(x) v_h(x) \ dx $$
  für alle Funktionen $v_h \in V_{h,0} = V_h \cap \{v(a)=v(b)=0\}$, d.h. alle Funktionen in $V_h$ die zusätzlich die Randbedingung $v(a)=v(b)=0$ erfüllen.  


- Äquivalent können wir fragen:

  Finde die Koeffizienten $c_j$, $j = 1,..,n-2$, sodass
  $$ \int_a^b k \frac{d u_h}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx = \int_a^b q_Q(x) \varphi_i(x) \ dx $$
  für alle $i = 1,..,n-2$.
 
 - D.h. wir haben $n-2$ Gleichungen für $n-2$ Unbekannte.


 * mit $u_h = T^a \cdot \varphi_0 (x) + \sum_{\color{red}{j=1}}^{\color{red}{n-2}} c_j \varphi_j(x) + T^b \cdot \varphi_{n-1} (x)$ können wir auch schreiben:
 
 Finde die Koeffizienten $c_j$, $j = 1,..,n-2$, sodass für alle $i = 1,..,n-2$ 
  $$ \sum_{{i=1}}^{{n-2}} c_j \underbrace{\int_a^b k \frac{d \varphi_j}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx}_{A_{i,j}} = f_i \qquad \text{mit}$$
  $$
  f_i = \int_a^b q_Q(x) \varphi_i(x) \ dx - T^a \int_a^b k \frac{d \varphi_0}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx - T^b \int_a^b k \frac{d \varphi_{n-1}}{d x}(x) \frac{d \varphi_i}{d x}(x) \ dx
  $$
  

 

$\leadsto$ Übungsaufgabe: Rechnen Sie den Schritt der letzten  Folien zu dieser nach.


 * Dies entspricht einer Matrix-Vektor-Gleichung
 $$
 \left( \begin{array}{cccc} 
 A_{1,1} & A_{1,2} & \dots & A_{1,n-2} \\ 
 A_{2,1} & A_{2,2} & \dots & A_{2,n-2} \\ 
 \vdots  & \vdots & \ddots & \vdots \\ 
 A_{n-2,1} & A_{n-2,2} & \dots & A_{n-2,n-2} \\ 
 \end{array}
 \right) \cdot 
  \left( \begin{array}{c} 
 c_{1} \\ c_2 \\ \vdots \\ c_{n-2}
 \end{array}
 \right)
 =
  \left( \begin{array}{c} 
 f_{1} \\ f_2 \\ \vdots \\ f_{n-2}
 \end{array}
 \right)
  $$
  
 * Lösung der linearen Gleichungen mit spezialisierten Lösern, z.B. basierend auf Gauss-Elimination 
 * Wie setzen sich nun die Matrix-Einträge zusammen?

#### Berechnung der Matrix-Einträge

In [ ]:
ComputeMatrixEntry(N=3,order=1,k=1)

#### "Dünn besetzte" System-Matrix: $A_{i,j} = 0$ für die meisten $(i,j)$-Paare

In [ ]:
align(interactive(Spy,N=[1,2,3,4,8,16],order=[1,2,3,4]))

#### Aufgabe:

* Betrachten wir $N = 3$, `order`$=1$ und $k=1$ und $q_Q=1$
* Es gibt $n=4$ Koeffizienten zu bestimmen
* Wir schreiben außerdem vor, dass der erste und der letzte Koeffizient 0 sind (Randbedingung)
* Bestimmen Sie die Matrixeinträge $A_{1,1}$, $A_{1,2}$, $A_{2,1}$, $A_{2,2}$ 
* Lösen Sie das Gleichungssystem
\begin{align*}
\begin{array}{rl}
  A_{1,1} c_1 + A_{1,2} c_2 &= f_1 \\
  A_{2,1} c_1 + A_{2,2} c_2 &= f_2 
\end{array} 
\qquad \Longleftrightarrow \qquad
\left( 
\begin{array}{rl}
  A_{1,1} & A_{1,2} \\
  A_{2,1} & A_{2,2} 
\end{array}
\right)
\cdot
\left(
\begin{array}{c} 
c_1 \\ 
c_2 
\end{array}
\right)=
\left(
\begin{array}{c} f_1 \\ f_2 \end{array} 
\right)
\end{align*}
 für die ermittelten Koeffizienten $A_{1,1}$, $A_{1,2}$, $A_{2,1}$, $A_{2,2}$ und
 $$
 f_1 = \int_{\Omega} \underbrace{q_Q}_{=1} \varphi_1 dx = h = 1/N = 1/3, \qquad
 f_2 = \int_{\Omega} \underbrace{q_Q}_{=1} \varphi_2 dx = h = 1/N = 1/3
 $$
* Skizzieren Sie die Lösung

## 4. Beispielrechnungen

 ![alt](heat1d.png) 

#### Die Differenzialgleichung:
$$
  - k(x) \frac{d^2 u}{d x^2}(x) = q_Q(x) \quad \text{für alle} \quad x \quad \text{ im Intervall } (a,b),
$$
wobei 
$$
  k(x) = \left\{ \begin{array}{cl} k_1 & \text{ für } x < 0.5 (a+b), \\ k_2 & \text{ sonst. }  \end{array} \right. , \qquad
  q_Q(x) = \left\{ \begin{array}{cl} q_Q^1 & \text{ für } x < 0.5(a+b), \\ q_Q^2 & \text{ sonst. }\end{array} \right.
$$
#### Mögliche Randbedinungen
 \begin{align*}
 \text{"Dirichlet":} & & u(a) &= T^a, &\qquad u(b) &= T^b, \\
 \text{"Neumann":} & &k \frac{d u}{dx}(a) &= q_S^a, &\qquad - k \frac{d u}{dx}(b) &= q_S^b, \\
 \text{"Robin":} & & k \frac{d u}{dx}(a) &= \alpha^a  (T^a - u(a)), & \qquad - k \frac{d u}{dx}(b) &= \alpha^b (T^b - u(b)), \\
 \end{align*}
 

In [ ]:
Heat1DExample()

#### Aufgabe:
Lösen Sie z.B. folgende Probleme (numerisch) und variieren sie $k_1,~k_2$ und $q_Q^1,~q_Q^2$:
* 1.:
 * Lösen Sie das Problem aus der letzten Aufgabe noch einmal numerisch und vergleichen Sie Ihr Ergebnis.
* 2.:
 * Am rechten Rand soll kein Wärmefluss übergehen
 * Am linken Rand wird eine Temperatur von 20°C gemessen
 * Auf $[0,1]$ wird komplett geheizt
 * Welche Temperatur liegt am rechten Rand vor?
* 3.:
 * Außerhalb von $[0,1]$ liegt eine Temperator von $\bar u = 20°C$ vor und es gilt das Wärmeübergangsgesetz:
   $$
     q_s \sim u - T^{\text{ref}}
   $$
 * Es wird auf dem ganzen Intervall geheizt
 * Die Wärmeleitfähigkeit ist rechts 10 mal so groß wie links
 

In [ ]:
Heat1DFEM( N=2, order=2, 
           k1 = 30, k2 = 30, 
           Q1 = 30000, Q2= 30000, 
           boundary_condition_left = "Dirichlet", boundary_condition_right = "Dirichlet", 
           value_left = 20, value_right = 20, 
           q_value_left = 0, q_value_right = 1, 
           r_value_left = 0, r_value_right = 1,
           intervalsize = 0.14)

### Wärmeverluste über die Mantelfläche

* Beziehen wir nun wieder Verluste über die Mantelfläche ein, so erhalten wir:
\begin{align*}
    - k \frac{d^2 u}{d x^2}(x) + \bar{\alpha} (u-\bar{T}) &= q_Q(x) \text{ in } (a,b)
\end{align*}
wobei $\bar{T}$ die (als konstant angenommene) Außentemperatur ist und $\bar{\alpha}$ der Wärmeübergangskoeffizient für den Wärmeübergang über die Mantelfläche.


* Das Problem zur (schwachen) Formulierung (FEM-Formulierung) mit Dirichlet-Randbed. wird damit:
Finde die Funktion $u(x)$ mit $u(a)=T^a$, $u(b)=T^b$, sodass für (fast) jede Funktion $v(x)$ mit $v(a)=v(b)=0$ gilt
$$ \int_a^b k \frac{d u}{d x}(x) + \bar{\alpha} u(x) v(x) \frac{d v}{d x}(x) \ dx = \int_a^b (q_Q(x)+\bar{\alpha} \bar{T}) v(x) \ dx. $$


* Entsprechende modifizierte linke und rechte Seite bei anderen Randbedingungen erhält man analog zum zuvor betrachteten Fall.


In [ ]:
Heat1DExample_with_ref()

In [ ]:
Heat1DFEM( N=200, order=1, 
           k1 = 50, k2 = 50, 
           Q1 = 30000, Q2= 30000, 
           boundary_condition_left = "Dirichlet", boundary_condition_right = "Dirichlet", 
           value_left = 20, value_right = 20, 
           q_value_left = 10, q_value_right = 1, 
           r_value_left = 0, r_value_right = 1,
           value_ref=20, c_ref = 80000,
           intervalsize = 0.14)